# Handle binary payloads from a Managed Online Endpoint

In this example, receiving and sending binary payloads in scoring scripts is demonstrated using the `rawhttp` decorator as well as the `AMLRequest` and `AMLResponse` objects. Without `rawhttp`, the run function is called passed the serialized JSON from the payload. Using `rawhttp`, the run function is instead passed an `AMLRequest` object, which wraps the native Flask request object used internally by the Azure Inference Server. After handling binary payloads, one can either return a JSON-serializable object as usual or use the `AMLResponse` object to have full control over the response, including returning binary payloads. 

## 1. Configure parameters, assets, and clients

### 1.1 Set workspace details

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

### 1.2 Set endpoint details

In [ ]:
import random

rand = random.randint(0, 10000)

endpoint_name = f"endpt-moe-bp-{rand}"

### 1.3 Set asset paths

In [ ]:
import os

base_path = "binary-payloads"
code_path = os.path.join(base_path, "code")
conda_file_path = os.path.join(base_path, "env.yml")
sample_data_path = os.path.join(base_path, "peacock.jpg")
output_data_path = os.path.join(base_path, "peacock-resized.jpg")

### 1.4 Download sample data

In [ ]:
import requests

url = "https://aka.ms/peacock-pic"
agent = f"Python Requests/{requests.__version__} (https://github.com/Azure/azureml-examples)"
r = requests.get(url, headers={"User-Agent": agent}, allow_redirects=True)
open(sample_data_path, "wb").write(r.content)

### 1.5 Create an MLClient Instance

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

## 2. Create endpoint

### 2.1 Define and create the endpoint

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
poller = ml_client.online_endpoints.begin_create_or_update(endpoint)
poller.wait()

### 2.2 Confirm that creation was successful

In [ ]:
from azure.ai.ml.exceptions import DeploymentException

status = poller.status()
if status != "Succeeded":
    raise DeploymentException(status)
else:
    print("Endpoint creation succeeded")
    endpoint = poller.result()
    print(endpoint)

## 3. Create a Binary-to-Binary Deployment

### 3.1 Examine the scoring script
This script receives an image as a binary file and returns a resized image as a binary file. Both scoring scripts use the `rawhttp` decorator to change the argument passed to the run function from JSON to the entire `AMLRequest` object. This script also uses the `AMLResponse` object 

```python
from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from azureml.contrib.services.aml_response import AMLResponse
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from PIL import Image
import io 

default_resize = (128, 128)

def init(): 
    pass 

@rawhttp
def run(req : AMLRequest):
    try:
        data = req.files.getlist("file")[0]
    except IndexError:
        return AMLResponse("No file uploaded", status_code=422)
    
    img = Image.open(data.stream)
    img = img.resize(default_resize)

    output = io.BytesIO()
    img.save(output, format="JPEG")
    resp = AMLResponse(message = output.getvalue(), status_code=200)
    resp.mimetype = "image/jpg"

    return resp
```

### 3.2 Create the deployment

In [ ]:
deployment = ManagedOnlineDeployment(
    name="binarypayloads",
    endpoint_name=endpoint_name,
    model=Model(path=base_path),
    code_configuration=CodeConfiguration(
        code=code_path, scoring_script="single-file-to-file-score.py"
    ),
    environment=Environment(
        conda_file=conda_file_path,
        image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
poller = ml_client.online_deployments.begin_create_or_update(deployment)
poller.wait()

### 3.3 Confirm the creation was successful


In [ ]:
status = poller.status()
if status != "Succeeded":
    raise DeploymentException(status)
else:
    print("Endpoint creation succeeded")
    deployment = poller.result()
    print(deployment)

### 3.4 Update endpoint traffic

In [ ]:
endpoint.traffic = {"binarypayloads": 100}
poller = ml_client.online_endpoints.begin_create_or_update(endpoint)
poller.wait()

## 4. Test the endpoint

### 4.1 Get endpoint details

In [ ]:
scoring_uri = endpoint.scoring_uri
key = ml_client.online_endpoints.get_keys(endpoint_name).primary_key

### 4.2 Send a request

In [ ]:
res = requests.post(
    url=scoring_uri,
    headers={"Authorization": f"Bearer {key}"},
    files=[("file", open(sample_data_path, "rb"))],
)
open(output_data_path, "wb").write(res.content)

## 5. Create a Binary-to-JSON Deployment

### 5.1 Examine the scoring script
This script accepts multiple image files uploaded as `file[]` and returns the sizes of the images as JSON. Both scoring scripts use the `rawhttp` decorator to change the argument passed to the run function from JSON to the entire `AMLRequest` object. However, unlike the first script this one returns a dictionary rather than an `AMLResponse` object.

```python
from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from PIL import Image

def init(): 
    pass 

@rawhttp
def run(req : AMLRequest):
    sizes = [{"filename" : f.filename,
        "size" : Image.open(f.stream).size}
        for f in req.files.getlist("file[]")]

    return {"response" : sizes}
```

### 5.2 Update the deployment

In [ ]:
deployment = ManagedOnlineDeployment(
    name="binarypayloads",
    endpoint_name=endpoint_name,
    model=Model(path=base_path),
    code_configuration=CodeConfiguration(
        code=code_path, scoring_script="multi-file-to-json-score.py"
    ),
    environment=Environment(
        conda_file=conda_file_path,
        image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
poller = ml_client.online_deployments.begin_create_or_update(deployment)
poller.wait()

### 5.3 Confirm the update was successful

In [ ]:
status = poller.status()
if status != "Succeeded":
    raise DeploymentException(status)
else:
    print("Deployment creation succeeded")
    deployment = poller.result()
    print(deployment)

## 6. Test the endpoint

### 6.1 Send a request

In [ ]:
res = requests.post(
    url=scoring_uri,
    headers={"Authorization": f"Bearer {key}"},
    files=[
        ("file[]", open(sample_data_path, "rb")),
        ("file[]", open(output_data_path, "rb")),
    ],
)

## 7. Delete assets

### 7.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)